In [1]:
import pandas as pd
import numpy as np

In [2]:
# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster=FA22&subject=CS" # get SP 22 CS classes
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())


In [4]:
classes = data_json['data']['classes']

In [5]:
url2 = 'https://classes.cornell.edu/api/2.0/config/subjects.json?roster=FA22'
response2 = urlopen(url2)
subjects_json = json.loads(response2.read())

In [6]:
subject_list = []
for i in subjects_json['data']['subjects']:
    subject_list.append(i['value'])


In [7]:
df = []
missedClasses = 0
cntr = 0
for subject in subject_list: 
    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=FA22&subject={subject}"
    response = urlopen(url)

    data_json = json.loads(response.read())

    classes = data_json['data']['classes']

    for class_ in classes:
        try:        
            obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]
            try:
                obj2 = obj['instructors'][0]
                prof = obj2['firstName'].strip() + ' ' + obj2['lastName'].strip()
            except:
                prof = ""

            startTime = obj['timeStart']
            endTime = obj['timeEnd']
            location = obj['facilityDescr']
            #x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], obj2['firstName'].strip() + ' ' + obj2['lastName'].strip(), startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]
            x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], prof, startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]

            df.append(x)
        except Exception as e:
            print(cntr)
            print(class_)
            print('\n')
            print(e)
            print('\n')
            missedClasses += 1
        cntr +=1
print(missedClasses)

430
{'strm': 2825, 'crseId': 354993, 'crseOfferNbr': 1, 'subject': 'ARCH', 'catalogNbr': '6308', 'titleShort': 'Sp Tp: Theory of Arch II', 'titleLong': 'Special Topics in the Theory of Architecture II', 'enrollGroups': [{'classSections': [{'ssrComponent': 'SEM', 'ssrComponentLong': 'Seminar', 'section': '101', 'classNbr': 3543, 'meetings': [], 'notes': [{'classNotesSeq': 1, 'descrlong': 'Prerequisite: ARCH 5301-ARCH 5302 or permission of instructor. Elective courses will be available during add/drop beginning in August.'}], 'campus': 'MAIN', 'campusDescr': 'Ithaca', 'location': 'ITH', 'locationDescr': 'Ithaca, NY (Main Campus)', 'startDt': '08/22/2022', 'endDt': '12/17/2022', 'addConsent': 'D', 'addConsentDescr': 'Department Consent Required', 'isComponentGraded': True, 'instructionMode': 'P', 'instrModeDescrshort': 'In Person', 'instrModeDescr': 'In Person', 'topicDescription': '', 'exploreCriteriaIds': [], 'materials': []}], 'unitsMinimum': 3, 'unitsMaximum': 3, 'componentsOptional':

In [9]:
df1 = pd.DataFrame(np.array(df))
df1.shape
#Sp22: 4544, FA22: 4162

(4162, 10)

In [10]:
df1 = df1.rename(columns={0:"Dept", 1:"Number", 2:"Course_Name", 3:"Professor", 4:"Start_Time", 5:"End_Time", 6:"Location",7:"Description",8:"Pre/coreqs",9:"Offered"})

In [11]:
median_grades = pd.read_csv("Cleaned_Medians.csv")

In [12]:
median_grades

,Unnamed: 0,Dept + Number,Professor,Median Grade,Semester (Ex: SP21),# of Students,Notes,Semester
0,1,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640",0.0
1,289,ENGL 1158 (FWS: Asian American Food Writing),Rhee,A,FA21,17,NaN,0.0
2,585,PLBIO 2300,Nixon,A,FA21,13,NaN,0.0
3,287,ENGL 1105,Various (FWS),A+,FA21,49,NaN,0.0
4,286,EDUC 2610,Steinhall,A,FA21,86,NaN,0.0
...,...,...,...,...,...,...,...,...
519,227,DEA 3510,Hedge,B,FA18,14,NaN,6.0
520,175,COMM 2200,Humphreys,A-,FA18,95,Cookie Cutter Essays,6.0
521,140,CEE 4510,Richardson,A-,FA17,44,NaN,8.0
522,171,COML 1109,Various (FWS),A-,FA17,NaN,NaN,8.0


In [13]:
median_grades_v2 = pd.read_csv("Cornell_Median_Grades_v2.csv")
median_grades_v2

,Dept + Number,Professor,Median Grade,Semester (Ex: SP21),# of Students,Notes
0,SEARCH BEFORE ADDING YOUR CLASS,NaN,NaN,NaN,NaN,NaN
1,AAS 2130,van Es,B+,SP21,239,NaN
2,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640"
3,AAS 2130,Moghimi,A,FA21,NaN,NaN
4,AEM 2010,Haeger,A-,FA21,138,NaN
...,...,...,...,...,...,...
637,VIEN 1104,Arnink/Moss,A,FA21,208,NaN
638,NaN,van Zuylen,B+,SP21,315,NaN
639,NaN,Packard/Karns,B+,FA18,441,NaN
640,NaN,Poitras,B,FA21,588,NaN


In [15]:
#UPDATE THIS IF NEEDED WHEN RE RUN
semesterdict = {"FA21":0, 'SP21':1, 'FA20':2, 'SP20':3, 'FA19':4,'SP19':5,'FA18':6,'SP18':7,'FA17':8,'SP17':9,'FA16':10}

In [16]:
#Data cleaning for medians
median_grades_v2['Semester'] = median_grades_v2['Semester (Ex: SP21)']
median_grades_v2 = median_grades_v2.replace({"Semester": semesterdict})
median_grades_v2 = median_grades_v2.sort_values(by="Semester")
median_grades_v2 = median_grades_v2.drop_duplicates(subset="Dept + Number", keep="first")
#median_grades_v2.to_csv("Cleaned_Medians.csv")

In [17]:
median_grades_v2 = median_grades_v2[['Dept + Number', 'Median Grade', 'Semester (Ex: SP21)', 'Semester']]

In [20]:
df1

,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered
0,AAS,2100,South Asian Diaspora,Viranjini Munasinghe,02:55PM,04:10PM,To Be Assigned,This interdisciplinary course (with an emphasi...,,Fall.
1,AAS,2130,Introduction to Asian American History,Derek Chang,11:25AM,12:40PM,To Be Assigned,"An introductory history of Chinese, Japanese, ...",,Fall.
2,AAS,3885,"Race and War in History: Workers, Soldiers, Pr...",Tejasvi Nagaraja,01:00PM,02:15PM,To Be Assigned,"Across twentieth-century history, race and war...",None,None
3,AAS,4040,Fictions of Dictatorship,Christine Balance,02:40PM,05:10PM,To Be Assigned,"Fictions of dictatorship, as termed by scholar...",,Fall.
4,AAS,4950,Independent Study,Viranjini Munasinghe,,,None,Independent reading course in topics not cover...,,"Fall, Spring."
...,...,...,...,...,...,...,...,...,...,...
4157,YORUB,2110,Intermediate Yoruba I,Adeolu Ademoyo,09:05AM,09:55AM,To Be Assigned,The intermediate course extends the developmen...,Prerequisite: YORUB 1109.,Fall.
4158,YORUB,3110,Advanced Yoruba I,Adeolu Ademoyo,07:30PM,09:25PM,To Be Assigned,This course will help students expand their un...,Prerequisite: YORUB 2111.,Fall.
4159,ZULU,1113,Elementary Zulu I,Angelika Kraemer,11:35AM,12:25PM,To Be Assigned,"A beginning course in conversational isiZulu, ...",,Fall.
4160,ZULU,2116,Intermediate Zulu I,Angelika Kraemer,09:25AM,10:15AM,To Be Assigned,"Development of fluency in speaking, listening,...",,Fall.


In [21]:
df1["Dept + Number"] = df1["Dept"] + ' ' + df1["Number"]

In [22]:
df1['Dept + Number'].value_counts()

AMST 4993     1
ANTHR 3110    1
MAE 6060      1
CRP 8900      1
PMA 7000      1
             ..
PHYS 1112     1
FDSC 4240     1
ART 3092      1
PMA 4222      1
HD 3330       1
Name: Dept + Number, Length: 4162, dtype: int64

In [23]:
df2 = df1.merge(median_grades, on='Dept + Number', how="outer")

In [24]:
df2 = df2.dropna(subset=['Dept'])

In [26]:
df2['Offered'].value_counts()

Fall.                                                                            2031
Fall, Spring.                                                                    1009
Spring.                                                                           312
Fall or Spring.                                                                   160
Fall, Spring, Summer.                                                             110
Fall, Summer.                                                                      57
Fall, Winter, Spring, Summer.                                                      39
Multi-semester course: (Fall, Spring).                                             24
Multi-semester course: (Fall).                                                      8
Fall, Winter, Summer.                                                               7
Summer.                                                                             4
Fall, spring.                                         

In [27]:
df2.to_csv("Classes_With_Medians_FA22.csv")